In [22]:
!pip install pandas scikit-learn numpy

In [23]:
import pandas as pd

train = pd.read_csv('train.csv')
oil = pd.read_csv('oil.csv')

In [24]:
def getOneStore(df, store_nbr):
    df = df[df['store_nbr'] == store_nbr]
    return df

train = getOneStore(train, 1)
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [25]:
def addRowWithSameDate(_to, _from, column):
    rows = _from['date']
    index = 0
    res = _to.copy()
    for row in rows:
        dateToFill = res[res['date'] == row].index
        for c in dateToFill:
            res.at[c, column] = _from.at[index, column]
        index += 1
    return res

updateTrain = addRowWithSameDate(train, oil, 'dcoilwtico')
updateTrain.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,NaN


In [26]:
holi_df = pd.read_csv('holidays_events.csv')
print(len(holi_df['date']))

In [27]:
def deleteTransferredTrue(df):
    if (df['transferred'].dtype == bool):
        x = 0
        for trans in df['transferred']:
            if trans:
                x += 1
        print('Number of transferred: ' + str(x))
        df = df.drop(df[df['transferred'] == True].index)
    return df

holi_df = deleteTransferredTrue(holi_df)

Number of transferred: 12


In [28]:
def addHolidayWithSameDate(_to, _from):
    rows = _from['date']
    for row in rows:
        dateToFill = _to[_to['date'] == row].index
        from_index = _from[_from['date'] == row].index[0]
        for c in dateToFill:
            _to.at[c, 'holiday'] = _from.at[from_index, 'locale_name']
    return _to

updateTrain = addHolidayWithSameDate(updateTrain, holi_df)

In [29]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(updateTrain['holiday'])
updateTrain['holiday'] = le.transform(updateTrain['holiday'])

In [30]:
updateTrain['dcoilwtico'] = updateTrain['dcoilwtico'].fillna(updateTrain['dcoilwtico'].median())

In [33]:
le.fit(updateTrain['family'])
updateTrain['family'] = le.transform(updateTrain['family'])

In [43]:
from sklearn.model_selection import train_test_split

X = updateTrain.drop(columns = ['sales', 'date'])
y = updateTrain['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

              id  store_nbr  family  onpromotion  dcoilwtico  holiday
0              0          1       0            0       53.33        4
1              1          1       1            0       53.33        4
2              2          1       2            0       53.33        4
3              3          1       3            0       53.33        4
4              4          1       4            0       53.33        4
...          ...        ...     ...          ...         ...      ...
2999134  2999134          1      28            0       47.57       19
2999135  2999135          1      29            0       47.57       19
2999136  2999136          1      30            7       47.57       19
2999137  2999137          1      31            0       47.57       19
2999138  2999138          1      32            0       47.57       19

[55572 rows x 6 columns]
0             0.000000
1             0.000000
2             0.000000
3             0.000000
4             0.000000
              ...  

In [45]:
from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(max_depth = 10)
clf = clf.fit(X_train, y_train)

In [46]:
predictions = clf.predict(X_test)
predictions

array([  5.30150309,   5.30150309,   3.47826087, ..., 747.86842105,
       923.65432099, 430.83835351])

In [48]:
from sklearn.metrics import r2_score
print(r2_score(predictions, y_test))

0.8871285715115442
